<a href="https://colab.research.google.com/github/MrVeGGi3/AI-for-Games/blob/main/PACMAN_SEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install durable-rules
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for durable-rules: filename=durable_rules-2.0.28-cp310-cp310-linux_x86_64.whl size=199325 sha256=be29517767562253e91dc8e9561f09b0b61368f0c3f6eb7050fb8cbae66a9bfb
  Stored in directory: /root/.cache/pip/wheels/f6/b4/6c/ee50ba2f212686ca9c6fdae66f5030f2f37cc9178a38503f69
Successfully built durable-rules


## Pac-Man

Comportamento geral dos fantasminhas:

Chase : Quando o Pac-Man está próximo e não está no modo de poder, os fantasminhas devem persegui-lo.

Flee : Quando o Pac-Man consome uma pílula de poder, os fantasminhas devem fugir.

Return to Base : Quando um fantasminha é comido, ele deve retornar à base.

eat_pac_man: Quando um fantasminha come o pac-man

observe: Quando o fantasminha observa o pac-man

### Regras


In [17]:
from durable.lang import *

with ruleset('ghosts'):
    # Regras para perseguir o Pac-Man
    @when_all((m.distance_to_pacman < 10) & (m.distance_to_pacman > 0) & (m.pacman_has_power == False) & (m.subject != 'Inky'))
    def chase(c):
      try:
        c.assert_fact({'subject': c.m.subject, 'action': 'move_towards', 'target': 'Pacman'})
        print(c.m)
        print(f'{c.m.subject} perseguindo o Pac-Man!')
      except Exception as e:
        print(f"ruleset: {e} duplicate")
    @when_all((m.subject == 'Inky') & (m.pac_man_fruits_collected > 50))
    def chase(c):
      try:
        c.assert_fact({'subject': c.m.subject, 'action': 'move_towards', 'target': 'Pacman'})
        print(c.m)
        print(f'{c.m.subject} perseguindo o Pac-Man!')
      except Exception as e:
        print(f"ruleset: {e} duplicate")

    # Regra para fugir do Pac-Man
    @when_all( (m.distance_to_pacman < 10) & (m.distance_to_pacman > 0) & (m.pacman_has_power == True))
    def flee(c):
      try:
        c.assert_fact({'subject': c.m.subject, 'action': 'move_away', 'target': 'Pacman'})
        print(c.m)
        print(f'{c.m.subject} fugindo do Pac-Man!!')
      except Exception as e:
        print(f"ruleset: {e} duplicate")

    # Regra para ficar com medo do Pac-Man
    @when_all(( m.pacman_has_power == True) & (m.distance_to_pacman > 0))
    def tremble(c):
      try:
        c.assert_fact({'subject': c.m.subject, 'action': 'move_away', 'target': 'Pacman'})
        print(c.m)
        print(f'{c.m.subject} Tremendo!')
      except Exception as e:
        print(f"ruleset: {e} duplicate")

    # Regra para retornar à base após ser comido
    @when_all((m.distance_to_pacman <= 0.01) & (m.distance_to_pacman >= -0.01) & (m.pacman_has_power == True))
    def return_to_base(c):
      try:
        c.assert_fact({'subject': c.m.subject, 'action': 'return_to', 'target': 'Base'})
        print(c.m)
        print(f'{c.m.subject} retornando a base!')
      except Exception as e:
        print(f"ruleset: {e} duplicate")

    #regra para comer o pac_man
    @when_all(m.distance_to_pacman <= 0)
    def eat_pac_man(c):
      try:
        print(f"ruleset: {e} duplicate")
        c.assert_fact({'subject': c.m.subject, 'action': 'return_to', 'target': 'Base'})
        print(c.m)
        print(f'{c.m.subject} comendo o pac-man!')
      except Exception as e:
        print(f"ruleset: {e} duplicate")

    # Regra para fantasma observar o pac-man (o que acontece sempre!)
    @when_all(+m.subject)
    def observe(c):
        print(f'{c.m.subject} observando o pac-man.')


##Inferência


In [4]:
post('ghosts', {'subject': 'Blinky', 'distance_to_pacman': 5, 'pacman_has_power': False})

{'subject': 'Blinky', 'distance_to_pacman': 5, 'pacman_has_power': False}
Blinky perseguindo o Pac-Man!
Blinky observando o pac-man.


{'sid': '0', 'id': 'sid-0', '$s': 1}

##Exercícios

a) Implemente as seguintes inferências

1.   Inky distante do pac-man em 20 unidades e pac-man com poder
2.   Blinky distante do pac-man em 3 unidades e pac-man com poder


3.   Pinky distante do pac-man em 10 unidades e pac-man coletou 60 frutas
4.   Inky distante do pac-man em 10 unidades e pac-man coletou 60 frutas









In [5]:
post('ghosts', {'subject': 'Inky', 'distance_to_pacman': 20, 'pacman_has_power' : True})
post('ghosts', {'subject' : 'Blinky', 'distance_to_pacman' : 20, 'pacman_has_power' : True})
post('ghosts', {'subject' : 'Pinky', 'distance_to_pacman' : 10, 'pacman_fruits_collected': 60})
post('ghosts', {'subject' : 'Inky', 'distance_to_pacman' : 10, 'pacman_fruits_collected' : 60})

{'subject': 'Inky', 'distance_to_pacman': 20, 'pacman_has_power': True}
Inky Tremendo!
Inky observando o pac-man.
{'subject': 'Blinky', 'distance_to_pacman': 20, 'pacman_has_power': True}
Blinky Tremendo!
Blinky observando o pac-man.
Pinky observando o pac-man.
Inky observando o pac-man.


{'sid': '0', 'id': 'sid-0', '$s': 1}

b) Execute as inferência a seguir e explique os resultados



In [6]:
post('ghosts', {'subject': 'Clyde', 'distance_to_pacman': 0 , 'pacman_has_power': True})

{'subject': 'Clyde', 'distance_to_pacman': 0, 'pacman_has_power': True}
Clyde retornando a base!
Clyde observando o pac-man.


{'sid': '0', 'id': 'sid-0', '$s': 1}

**Resposta:** Clyde entrou em colisão com o PacMan quando este capturou a bolinha. Logo, ele retorna a base, e volta de novo á observar o pac-man, esperando por uma distância minima para agir.


In [7]:
post('ghosts', {'subject': 'Clyde', 'eaten': True})

Clyde observando o pac-man.


{'sid': '0', 'id': 'sid-0', '$s': 1}



```
# Isto está formatado como código
```
**Resposta:** Como não existe a declaração de 'eaten' no @when_all, o programa retorna o método "observe".

In [8]:
post('ghosts', {'subject': 'Clyde', 'distance_to_pacman': 0})

ruleset: local variable 'e' referenced before assignment duplicate


{'sid': '0', 'id': 'sid-0', '$s': 1}

**Resposta:** O programa identifica que o Fantasma já foi chamado anteriormente, e portanto, é uma duplicata.

c) Elabore 3 inferências para demonstrar o comportamento dos fantasminhas em condições próximas dos limites para atacar o pac-man e outras 3 nos limites para retornar à base

In [9]:
post('ghosts', {'subject' : 'Clyde', 'distance_to_pacman': 0.5,})

Clyde observando o pac-man.


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [10]:
post('ghosts',{'subject' : 'Inky', 'distance_to_pacman' : 10.001})

Inky observando o pac-man.


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [11]:
post('ghosts', {'subject' : 'Clyde', 'distance_to_pacman': 0.0001})

Clyde observando o pac-man.


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [13]:
post('ghosts', {'subject' : 'Inky', 'distance_to_pacman': 0.0000000001, 'pacman_has_power' : True})

error: Could not assert event, error code: 204

In [14]:
post('ghosts', {'subject' : 'Inky', 'distance_to_pacman': 0.01, 'pacman_has_power' : True})

ruleset: {'subject': 'Inky', 'action': 'move_away', 'target': 'Pacman', 'sid': '0'} duplicate


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [ ]:
post('ghosts', {'subject' : 'Clyde', 'distance_to_pacman': 0.1, 'pacman_has_power' : True})

d) Identifique e implemente regras para o comportamento dos fantasmas não previstas na base "ghosts" . Implemente inferências para testar as regras propostas

**Tentativa 1**: Implementar uma range com maior precisão de casas decimais no retornar á base. Impedindo com que se confunda a chamada de função do "Ir á base" e "Fugir"

In [20]:
post('ghosts', {'subject' : 'Inky', 'distance_to_pacman': 0, 'pacman_has_power' : True})

{'subject': 'Inky', 'distance_to_pacman': 0, 'pacman_has_power': True}
Inky retornando a base!
Inky observando o pac-man.


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [30]:
post('ghosts', {'subject' : 'Clyde', 'distance_to_pacman': 0.5, 'pacman_has_power' : True})

ruleset: {'subject': 'Clyde', 'action': 'move_away', 'target': 'Pacman', 'sid': '0'} duplicate


{'sid': '0', 'id': 'sid-0', '$s': 1}